# Machine Learning: Exercise session 04

In this exercise session we will focus on Ridge and Lasso regression. You will learn how to fit, predict, and cross-validate these models.

In the first problem, we will continue using the housing dataset where we added several interaction terms, and one additional variable named `X1`. You can download the data from Moodle in this week's section.

The second problem is theoretical and not due for hand-in. There, we will derive the closed-form solution for the Ridge regression (notice that Lasso regression has no closed-form solution).

## Problem 1

### 0. Load data and create a test set

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

* Import the clean housing dataset with `pd.read_csv` and take a quick look at it, to verify that it is in the desired shape (display the dataframe, check its "infos").

In [3]:
df=pd.read_csv("housing_clean.csv")

In [5]:
df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20433 entries, 0 to 20432
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   longitude             20433 non-null  float64
 1   latitude              20433 non-null  float64
 2   housing_median_age    20433 non-null  float64
 3   median_income         20433 non-null  float64
 4   median_house_value    20433 non-null  float64
 5   rooms_per_bedroom     20433 non-null  float64
 6   rooms_per_household   20433 non-null  float64
 7   people_per_household  20433 non-null  float64
 8   ocean_proximity_enc   20433 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


* Separate the dataframe into the features `X` and the target variable `y`: (remember, we want to predict the median house value, given the other variables)

In [6]:
X = df[["longitude","latitude", "housing_median_age","median_income","rooms_per_bedroom","rooms_per_household","people_per_household","ocean_proximity_enc"]]
Y = df[["median_house_value"]]

* Using the function `train_test_split`, split the dataset into training and test set. Set `test_size = 0.95` and `random_state = 12`. 

*Notice that we choose a very large fraction of test data because we want to see whether the Ridge and Lasso regression can handle the high-dimensional setting.*

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(X,Y, test_size=.95, random_state=12)

### 1. Define models

* Using the class `Ridge` from `sklearn.linear_model`, instantiate a ridge regression object with penalty parameter equal to 0.1. Furthermore, set `fit_intercept=True`, and `normalize=True`. Call this object `ridge_reg`.

In [14]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.1, fit_intercept=True, normalize= True)


* Using the class `Lasso` from `sklearn.linear_model`, instantiate a ridge regression object with penalty parameter equal to 0.1. Furthermore, set `fit_intercept=True`, `normalize=True`, and `max_iter=1e5`. Call this object `lasso_reg`.

In [16]:
from sklearn.linear_model import Lasso
lasso_reg=Lasso(alpha=0.1, fit_intercept= True, normalize = True, max_iter=1e5)

What is the role of the option `fit_intercept`?
*The idea is too ask lasso to add the beta0 (the intercept) to its penality term*.

### 2. Fit Ridge regression

* Fit the ridge regression on the training data previously created.

In [17]:
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.1, normalize=True)

* Compute the root mean square error of the fitted model on both the training and test set. What do you observe?
Plus grand erreur de prédiction sur le testing data set que le training. Mais pas égal à 0 pour le training, car ridge empêche overfitting.

In [22]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, ridge_reg.predict(x_train), squared=False)

70989.23791611486

In [23]:
mean_squared_error(y_test, ridge_reg.predict(x_test), squared=False)

106065.35468065333

In [24]:
# R^2: 1 - E[(Y - Y_hat)^2]/E[(Y - E[Y])^2] 
# (proportion of variability of Y explained by fitted model)
1 - mean_squared_error(y_test, ridge_reg.predict(x_test)) / np.var(y_test)

median_house_value    0.154895
dtype: float64

### 4. Plot Ridge regression coefficients

We now want to plot the coefficients of the ridge regression for different values of the penalty parameter.

* Create a grid, named `grid`, of penalty values ranging from `1e-4` to `1e4`.

In [37]:
from sklearn.model_selection import GridSearchCV, KFold
grid = {"alpha" : np.arange(1e-4, 1e4, 100)}
grid["alpha"]

array([1.0000000e-04, 1.0000010e+02, 2.0000010e+02, 3.0000010e+02,
       4.0000010e+02, 5.0000010e+02, 6.0000010e+02, 7.0000010e+02,
       8.0000010e+02, 9.0000010e+02, 1.0000001e+03, 1.1000001e+03,
       1.2000001e+03, 1.3000001e+03, 1.4000001e+03, 1.5000001e+03,
       1.6000001e+03, 1.7000001e+03, 1.8000001e+03, 1.9000001e+03,
       2.0000001e+03, 2.1000001e+03, 2.2000001e+03, 2.3000001e+03,
       2.4000001e+03, 2.5000001e+03, 2.6000001e+03, 2.7000001e+03,
       2.8000001e+03, 2.9000001e+03, 3.0000001e+03, 3.1000001e+03,
       3.2000001e+03, 3.3000001e+03, 3.4000001e+03, 3.5000001e+03,
       3.6000001e+03, 3.7000001e+03, 3.8000001e+03, 3.9000001e+03,
       4.0000001e+03, 4.1000001e+03, 4.2000001e+03, 4.3000001e+03,
       4.4000001e+03, 4.5000001e+03, 4.6000001e+03, 4.7000001e+03,
       4.8000001e+03, 4.9000001e+03, 5.0000001e+03, 5.1000001e+03,
       5.2000001e+03, 5.3000001e+03, 5.4000001e+03, 5.5000001e+03,
       5.6000001e+03, 5.7000001e+03, 5.8000001e+03, 5.9000001e

* Fill in the `??` below.

In [38]:
coefs = []
for alpha in grid:
    ridge_reg.set_params(alpha= alpha)
    ridge_reg.fit(x_train, y_train)
    coefs.append(ridge_reg.coef_[0])

ValueError: could not convert string to float: 'alpha'

In [ ]:
plt.plot(np.log10(??), ??);

### 5. Fit Lasso regression

* Fit the lasso regression on the training data previously created.

* Compute the root mean square error of the fitted model on both the training and test set. What do you observe?

### 6. Plot Lasso coefficients

As above, we want to plot the coefficients of the lasso regression for different values of the penalty parameter.

* Fill in the `??` below.

In [ ]:
coefs = []
for ?? in grid:
    lasso_reg.set_params(??=??)
    lasso_reg.fit(??, ??)
    coefs.append(lasso_reg.??)

In [ ]:
plt.plot(np.log10(??), ??);

### 7. Prepare CV

At this point, we want to find the optimal penalty parameter for our models.
We will use 10-fold cross-validation.

* Using the `KFold` class from `sklearn.model_selection` and instatiate an object named `folds`. Set the number of splits equal to 10, the random seed equal to 42, and make sure to shuffle the rows.

### 8. CV for Ridge

* Define a grid of penalty parameters for the ridge regression. You can use the same values used for the grid above. Name the object `ridge_grid`.

* Create a dictionary with the key-value pair `"alpha"`, `ridge_grid`. What is the use of this object?

* Import the class `GridSearchCV` from `sklearn.model_selection` and use it to instantiate a **cross-validation object** for the ridge regression. Make sure to include the following parameters: `estimator`, `param_grid`, `scoring`, `cv`.

* Run the cross-validation by calling the `fit` method of the **cross-validation object** that you created at the point before.

* We want to plot the cross-validation error against different values of the penalty parameter. Fill in the `??`.

In [ ]:
# Choose best model according to 1-se rule
mean_scores = -??.cv_results_["mean_test_score"]
std_scores = ??.cv_results_["std_test_score"]
alphas = ??.cv_results_["param_alpha"].data

best_index = ??
min_alpha_ridge = ??
print("Minimum alpha:", min_alpha_ridge)

In [ ]:
plt.errorbar(x=np.log10(??), y=??, yerr = ??, fmt='o', capsize=3)

plt.axvline(np.log10(??), ls='dotted', color="grey")#vertical line at the k yielding minimum CV MSE

plt.title("Ridge regressor CV error")
plt.xlabel('log(lambda)')
plt.ylabel('Mean Squared Error')
plt.show()

### 9. CV for Lasso

* Define a grid of penalty parameters for the lasso regression. You can use the same values used for the grid above. Name the object `lasso_grid`.

* Create a dictionary with the key-value pair `"alpha"`, `lasso_grid`. What is the use of this object?

* Import the class `GridSearchCV` from `sklearn.model_selection` and use it to create a **cross-validation object** for the lasso regression. Make sure to include the following parameters: `estimator`, `param_grid`, `scoring`, `cv`.

* Run the cross-validation by calling the `fit` method of the **cross-validation object** that you created at the point before.

* We want to plot the cross-validation error against different values of the penalty parameter. Fill in the `??`.

In [ ]:
# Choose best model according to 1-se rule
mean_scores = -??.cv_results_["mean_test_score"]
std_scores = ??.cv_results_["std_test_score"]
alphas = ??.cv_results_["param_alpha"].data

best_index = ??
min_alpha_lasso = ??
print("Minimum alpha:", min_alpha_lasso)

In [ ]:
plt.errorbar(x=np.log10(??), y=??, yerr = ??, fmt='o', capsize=3)

plt.axvline(np.log10(??), ls='dotted', color="grey")#vertical line at the k yielding minimum CV MSE

plt.title("Lasso regressor CV error")
plt.xlabel('log(lambda)')
plt.ylabel('Mean Squared Error')
plt.show()

### 10. Compute performance of the two best models

* Given the optimal tuning parameter of the ridge and lasso regression, refit both models on the entire training data set. Furthermore, compute their root mean square error on the test set. How do they compare to the errors obtained before performing the cross-validation?

_Hint_: As optimal tuning parameter, choose the one that minimizes the Mean Squared Error and not the one according to the 1-standard error rule.

In [ ]:
# Refit ridge on training and evaluate on test


In [ ]:
# Refit lasso on training and evaluate on test


## Problem 2

In this problem, you are asked to derive the closed-form solution for the ridge regression coefficients.

Let $\mathbf{X}\in\mathbb{R}^{n\times p}$ denote the matrix of predictors, and let $\mathbf{y}\in\mathbb{R}^n$ denote the target vector.

The optimal ridge regression coefficient vector $\beta^*$, with parameter $\lambda > 0$, is defined as

$$\beta^* := \arg\min_{\beta\in\mathbb{R}^p} \ (\mathbf{y} - \mathbf{X}\beta)^T(\mathbf{y} - \mathbf{X}\beta) + \lambda \beta ^T\beta.$$

Show that 


$$\beta^*  = (\mathbf{X}^T\mathbf{X} + \lambda \mathbf{I}_p)^{-1} \mathbf{X}^T \mathbf{y},$$

where $\mathbf{I}_p$ denote the identity matrix of size $p\times p$.